In [1]:
import os
os.getcwd()


'/Users/nehaagarwal/Desktop/projects/Text_Summarizer_Project/research'

In [2]:
os.chdir("../.")
os.getcwd()

'/Users/nehaagarwal/Desktop/projects/Text_Summarizer_Project'

In [17]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen = True)
class ModelTrainerConfig:
    root_dir : Path
    data_path: Path
    model_ckpt : Path
    num_train_epochs : int
    warmup_steps : int
    per_device_train_batch_size: int
    per_device_eval_batch_size : int
    weight_decay : float
    logging_steps : int
    evaluation_strategy : str
    eval_steps : int
    save_steps : float
    gradient_accumulation_steps : int

In [18]:
from textsummarizer.constant import *
from textsummarizer.utils.common import read_yaml,create_directories

In [19]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath=CONFIG_FILE_PATH,
                 params_filepath=PARAM_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.TrainingArgumants

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir = config.root_dir,
            data_path = config.data_path,
            model_ckpt = config.model_ckpt,
            num_train_epochs = params.num_train_epochs,
            warmup_steps = params.warmup_steps,
            per_device_train_batch_size = params.per_device_train_batch_size,
            per_device_eval_batch_size = params.per_device_eval_batch_size,
            weight_decay = params.weight_decay,
            logging_steps = params.logging_steps,
            evaluation_strategy = params.evaluation_strategy,
            eval_steps = params.eval_steps,
            save_steps = params.save_steps,
            gradient_accumulation_steps = params.gradient_accumulation_steps
            
        )
        return model_trainer_config

In [20]:
from transformers import AutoModelForSeq2SeqLM,AutoTokenizer
from datasets import load_dataset,load_from_disk,load_metric
import torch
from transformers import DataCollatorForSeq2Seq
from transformers import TrainingArguments,Trainer

In [30]:
class ModelTrainer:
    def __init__(self,config:ModelTrainerConfig):
        self.config = config

    def train(self):
        device = device = torch.device('mps')
        tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt).to(device)
        seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_pegasus)

        dataset = load_from_disk(self.config.data_path)

        
        # trainer_args = TrainingArguments(
        #     output_dir = self.config.root_dir,
        #     num_train_epochs = self.config.num_train_epochs,
        #     warmup_steps = self.config.warmup_steps,
        #     per_device_train_batch_size = self.config.per_device_train_batch_size,
        #     per_device_eval_batch_size = self.config.per_device_eval_batch_size,
        #     weight_decay = self.config.weight_decay,
        #     logging_steps = self.config.logging_steps,
        #     evaluation_strategy = self.config.evaluation_strategy,
        #     eval_steps = self.config.eval_steps,
        #     save_steps = self.config.save_steps,
        #     gradient_accumulation_steps = self.config.gradient_accumulation_steps)


        trainer_args = TrainingArguments(
            output_dir=self.config.root_dir, 
            num_train_epochs= 1, 
            warmup_steps=500,
            per_device_train_batch_size=1, 
            per_device_eval_batch_size=1,
            weight_decay=0.01, 
            logging_steps=10,
            evaluation_strategy='steps', 
            eval_steps=500, 
            save_steps=1e6,
            gradient_accumulation_steps=16
        ) 

        trainer = Trainer(
            model=model_pegasus,
            args = trainer_args,
            tokenizer = tokenizer,
            data_collator = seq2seq_data_collator,
            train_dataset = dataset['test'][:10],
            eval_dataset = dataset['validation'][:5])
        
        trainer.train()
        model_pegasus.save_pretrained(os.path.join(self.config.root_dir,"pegasus-samsum-model"))
        tokenizer.save_pretrained(os.path.join(self.config.root_dir,"tokenizer"))

    



In [31]:
!export MallocStackLogging=1
!export PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.1

Python(88878) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(88879) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


In [32]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer = ModelTrainer(config = model_trainer_config)
    model_trainer.train()
except Exception as e:
    raise e

[2024-01-21 17:23:42,765: INFO: common: yaml file:config/config.yaml loaded successfully]
[2024-01-21 17:23:42,769: INFO: common: yaml file:params.yaml loaded successfully]
[2024-01-21 17:23:42,770: INFO: common: directory:artifacts already exists]
[2024-01-21 17:23:42,771: INFO: common: directory:artifacts/model_trainer already exists]


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RuntimeError: MPS backend out of memory (MPS allocated: 8.94 GB, other allocations: 133.77 MB, max allowed: 9.07 GB). Tried to allocate 4.00 KB on private pool. Use PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0 to disable upper limit for memory allocations (may cause system failure).